import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']

my_amazon_product_reviews = mydb['amazon_product_reviews']

In [23]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
#from nltk import word_tokenize
import string
from nltk.corpus import stopwords
            
            
class MySentences():
    def __init__(self):
        self.reviews_cursor = my_amazon_product_reviews.find()
        self.sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        self.stop = set(stopwords.words('english'))
        self.exclude = set(string.punctuation)
        self.lemma = WordNetLemmatizer()
 
    def __iter__(self):
        for review_item in self.reviews_cursor:
            reviewText = review_item["reviewText"]
            for sentence in self.sent_detector.tokenize(reviewText):
                #print(sentence)
                tokens = nltk.word_tokenize(sentence.lower())
                #tokens = word_tokenize(sentence.lower())
                #tokens = sentence.lower().split()
                normalized_tokens = [self.lemma.lemmatize(token) 
                                     for token in tokens 
                                     #if token not in self.stop and 
                                     token not in self.exclude
                                    ]
                print(normalized_tokens)
                yield normalized_tokens

In [ ]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
#from nltk import word_tokenize
import string
from nltk.corpus import stopwords
import gensim
            
            
class MySentences():
    def __init__(self):
        #self.reviews_cursor = my_amazon_product_reviews.find()
        self.sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        self.stop = set(stopwords.words('english'))
        self.exclude = set(string.punctuation)
        self.lemma = WordNetLemmatizer()
 
    def __iter__(self):
        for review_item in parse("../../download/reviews_Cell_Phones_and_Accessories.json.gz"): 
        #for review_item in self.reviews_cursor:
            reviewText = review_item["reviewText"]
            s_i = 0
            for sentence in self.sent_detector.tokenize(reviewText):
                s_i = s_i + 1
                #print(sentence)
                tokens = nltk.word_tokenize(sentence.lower())
                #tokens = word_tokenize(sentence.lower())
                #tokens = sentence.lower().split()
                normalized_tokens = [self.lemma.lemmatize(token) 
                                     for token in tokens 
                                     #if token not in self.stop and 
                                     if token not in self.exclude
                                    ]
                #print(normalized_tokens)
                label = review_item["asin"] + "_" + review_item["reviewerID"] + "_" + str(s_i)
                yield gensim.models.doc2vec.LabeledSentence(words=normalized_tokens, tags=[label])
                
    

In [ ]:

sentences = MySentences()
model = gensim.models.Doc2Vec(sentences)

In [ ]:
model.save('../../download/myamazon_doc2vec_model')